In [2]:
from google.colab import files
uploaded = files.upload()
import pandas as pd
#from google.colab import drive
#drive.mount('/content/gdrive')
#root_path = 'gdrive/My Drive/Colab Notebooks/NLP/NER/'
data = pd.read_csv('ner_dataset.csv', encoding= 'unicode_escape')
data.head()

Saving ner_dataset.csv to ner_dataset (1).csv


,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [9]:
#extracting the mappings required to train the neural network
from itertools import chain
def get_dict_map(data, token_or_tag):
    tok2idx = {}
    idx2tok = {}
    
    #Getting unique values of words and tags
    if token_or_tag == 'token':
        vocab = list(set(data['Word'].to_list()))
    else:
        vocab = list(set(data['Tag'].to_list()))
    #Create 2 dictionaries {value, index} and {index, value} for all the items in the vocab 
    idx2tok = {idx:tok for  idx, tok in enumerate(vocab)}
    tok2idx = {tok:idx for  idx, tok in enumerate(vocab)}
    return tok2idx, idx2tok
token2idx, idx2token = get_dict_map(data, 'token')
tag2idx, idx2tag = get_dict_map(data, 'tag')

In [14]:
#transform the columns in the data to extract the sequential data for neural network
#Get the sequential indexes of all the words and the tags
data['Word_idx'] = data['Word'].map(token2idx)
data['Tag_idx'] = data['Tag'].map(tag2idx)
#method='ffill': Ffill or forward-fill propagates the last observed non-null value forward until another non-null value is encountered
data_fillna = data.fillna(method='ffill', axis=0)
data_fillna[1:25]

,Sentence #,Word,POS,Tag,Word_idx,Tag_idx
1,Sentence: 1,of,IN,O,4938,5
2,Sentence: 1,demonstrators,NNS,O,931,5
3,Sentence: 1,have,VBP,O,19137,5
4,Sentence: 1,marched,VBN,O,14871,5
5,Sentence: 1,through,IN,O,9561,5
6,Sentence: 1,London,NNP,B-geo,2174,17
7,Sentence: 1,to,TO,O,6442,5
8,Sentence: 1,protest,VB,O,8166,5
9,Sentence: 1,the,DT,O,3414,5
10,Sentence: 1,war,NN,O,16068,5


In [15]:
data_group = data_fillna.groupby(['Sentence #'],as_index=False)['Word', 'POS', 'Tag', 'Word_idx', 'Tag_idx'].agg(lambda x: list(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [19]:
data_group


,Sentence #,Word,POS,Tag,Word_idx,Tag_idx
0,Sentence: 1,"[Thousands, of, demonstrators, have, marched, ...","[NNS, IN, NNS, VBP, VBN, IN, NNP, TO, VB, DT, ...","[O, O, O, O, O, O, B-geo, O, O, O, O, O, B-geo...","[14701, 4938, 931, 19137, 14871, 9561, 2174, 6...","[5, 5, 5, 5, 5, 5, 17, 5, 5, 5, 5, 5, 17, 5, 5..."
1,Sentence: 10,"[Iranian, officials, say, they, expect, to, ge...","[JJ, NNS, VBP, PRP, VBP, TO, VB, NN, TO, JJ, J...","[B-gpe, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[14286, 20013, 15400, 5817, 18056, 6442, 15860...","[6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ..."
2,Sentence: 100,"[Helicopter, gunships, Saturday, pounded, mili...","[NN, NNS, NNP, VBD, JJ, NNS, IN, DT, NNP, JJ, ...","[O, O, B-tim, O, O, O, O, O, B-geo, O, O, O, O...","[15080, 8583, 3725, 1034, 9752, 2805, 18069, 3...","[5, 5, 10, 5, 5, 5, 5, 5, 17, 5, 5, 5, 5, 5, 1..."
3,Sentence: 1000,"[They, left, after, a, tense, hour-long, stand...","[PRP, VBD, IN, DT, NN, JJ, NN, IN, NN, NNS, .]","[O, O, O, O, O, O, O, O, O, O, O]","[5877, 14552, 22277, 2321, 20946, 1625, 19925,...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]"
4,Sentence: 10000,"[U.N., relief, coordinator, Jan, Egeland, said...","[NNP, NN, NN, NNP, NNP, VBD, NNP, ,, NNP, ,, J...","[B-geo, O, O, B-per, I-per, O, B-tim, O, B-geo...","[12049, 22416, 13613, 5749, 18505, 13914, 1479...","[17, 5, 5, 4, 7, 5, 10, 5, 17, 5, 6, 5, 6, 5, ..."
...,...,...,...,...,...,...
21980,Sentence: 9995,"[Opposition, leader, Mir, Hossein, Mousavi, ha...","[NNP, NN, NNP, NNP, NNP, VBZ, VBN, PRP, VBZ, T...","[O, O, O, B-per, I-per, O, O, O, O, O, O, O, O...","[4090, 9997, 21797, 12284, 1283, 10829, 13914,...","[5, 5, 5, 4, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ..."
21981,Sentence: 9996,"[On, Thursday, ,, Iranian, state, media, publi...","[IN, NNP, ,, JJ, NN, NNS, VBN, DT, NN, IN, DT,...","[O, B-tim, O, B-gpe, O, O, O, O, O, O, O, O, B...","[16341, 16336, 3820, 14286, 20635, 13878, 1767...","[5, 10, 5, 6, 5, 5, 5, 5, 5, 5, 5, 5, 16, 12, ..."
21982,Sentence: 9997,"[Following, Iran, 's, disputed, June, 12, elec...","[VBG, NNP, POS, JJ, NNP, CD, NNS, ,, NNS, NNS,...","[O, B-geo, O, O, B-tim, I-tim, O, O, O, O, O, ...","[684, 23108, 6578, 13578, 15273, 8692, 22555, ...","[5, 17, 5, 5, 10, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5..."
21983,Sentence: 9998,"[Since, then, ,, authorities, have, held, publ...","[IN, RB, ,, NNS, VBP, VBN, JJ, NNS, IN, DT, VB...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[6386, 7633, 3820, 8582, 19137, 9004, 7920, 36...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ..."


In [20]:
'''We are using LSTM for training the model. LSTM layers accept sequences of the same length only.
So every sentence that appears as integer in the data must be padded with the same length (Using Keras Pre-processing).
And splitting the data into train, test and validate datasets
'''
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

def get_pad_train_test_val(data_group, data):

    #get token and tag length
    n_token = len(list(set(data['Word'].to_list()))) #Unique number of words in the data
    n_tag = len(list(set(data['Tag'].to_list()))) #Unique number of tags in the data

    #Pad tokens (X var)    
    tokens = data_group['Word_idx'].tolist() #List of lists of tokens
    maxlen = max([len(s) for s in tokens]) # Maximum length of the list in the list
    pad_tokens = pad_sequences(tokens, maxlen=maxlen, dtype='int32', padding='post', value= n_token - 1) #Pad given value after the list(post) 

    #Pad Tags (y var) and convert it into one hot encoding
    tags = data_group['Tag_idx'].tolist() #List of lists of tags
    pad_tags = pad_sequences(tags, maxlen=maxlen, dtype='int32', padding='post', value= tag2idx["O"]) #Default padding value O (same length as tokens)
    n_tags = len(tag2idx) #Get unique categories in tags column
    pad_tags = [to_categorical(i, num_classes=n_tags) for i in pad_tags] #Create additional columns for each category (One-hot encoding)
    
    #Split train, test and validation set
    tokens_, test_tokens, tags_, test_tags = train_test_split(pad_tokens, pad_tags, test_size=0.1, train_size=0.9, random_state=2020)
    train_tokens, val_tokens, train_tags, val_tags = train_test_split(tokens_,tags_,test_size = 0.25,train_size =0.75, random_state=2020)

    print(
        'train_tokens length:', len(train_tokens),
        '\ntrain_tags length:', len(train_tags),
        '\ntest_tokens length:', len(test_tokens),
        '\ntest_tags:', len(test_tags),
        '\nval_tokens:', len(val_tokens),
        '\nval_tags:', len(val_tags),
    )
    
    return train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags

train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags = get_pad_train_test_val(data_group, data)

train_tokens length: 14839 
train_tags length: 14839 
test_tokens length: 2199 
test_tags: 2199 
val_tokens: 4947 
val_tags: 4947


In [21]:
#Training Neural Network for Named Entity Recognition (NER)

import numpy as np
import tensorflow
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.utils import plot_model
from numpy.random import seed
seed(1)
tensorflow.random.set_seed(2)

In [22]:
def lstm_model():
    model = Sequential()
    # Add Embedding layer
    model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_length))
    # Add bidirectional LSTM
    model.add(Bidirectional(LSTM(units=output_dim, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode = 'concat'))
    # Add LSTM
    model.add(LSTM(units=output_dim, return_sequences=True, dropout=0.5, recurrent_dropout=0.5))
    # Add timeDistributed Layer
    model.add(TimeDistributed(Dense(n_tags, activation="relu")))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model

In [23]:
def train_model(X, y, model):
    loss = list()
    for i in range(25):
        # fit model for one epoch on this sequence
        hist = model.fit(X, y, batch_size=1000, verbose=1, epochs=1, validation_split=0.2)
        loss.append(hist.history['loss'][0])
    return loss

In [24]:
#Specify the input and output dimensions to the neural network
input_dim = len(list(set(data['Word'].to_list())))+1
output_dim = 64
input_length = max([len(s) for s in data_group['Word_idx'].tolist()])
n_tags = len(tag2idx)

In [25]:
results = pd.DataFrame()
model = lstm_model()
plot_model(model)
results['with_add_lstm'] = train_model(train_tokens, np.array(train_tags), model)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 72, 64)            1547776   
_________________________________________________________________
bidirectional (Bidirectional (None, 72, 128)           66048     
_________________________________________________________________
lstm_1 (LSTM)                (None, 72, 64)            49408     
_________________________________________________________________
time_distributed (TimeDistri (None, 72, 18)            1170      
Total params: 1,664,402
Trainable params: 1,664,402
Non-trainable params: 0
_________________________________________________________________
12/12 [==============================] - 43s 4s/step - loss: 0.2008 - accuracy: 0.9538 - val_loss: 0.2214 - val_accuracy: 0.9534


In [27]:
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')
text = nlp('Hi, I am Manvitha \n I live in Ireland \n I want to work for Facebook \n Obama is My Inspiration')
displacy.render(text, style = 'ent', jupyter=True)